In [ ]:
] add CSV

In [ ]:
] add DataFrames

In [ ]:
using CSV

In [ ]:
using DataFrames

In [ ]:
alldata = CSV.read("data.csv", DataFrame)

In [ ]:
alldata.x

In [ ]:
size(alldata)

In [ ]:
size(alldata,1)

In [ ]:
data = alldata[5:size(alldata,1), :]

In [ ]:
using WGLMakie

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :black)
f

In [ ]:
m = 2
b = 100

In [ ]:
xx = LinRange(50, 250, 25)

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :black)
yy = m .*xx .+ b
lines!(xx, yy)
f

In [ ]:
] add Optim

In [ ]:
using Optim

In [ ]:
function abs_diff(x, mean)
    return sum(abs.(x .- mean))
end

In [ ]:
function line_abs_diff(x, b, m, y)
    y_pred = b .+ x .* m
    return abs_diff(y_pred, y)
end

In [ ]:
function objective_abs(params)
    b = params[1]
    m = params[2]
    return line_abs_diff(data.x, b, m, data.y)
end

In [ ]:
result = optimize(objective_abs, [b,m] .+ 0.)

In [ ]:
b_abs,m_abs = Optim.minimizer(result)

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :black)
lines!(xx, b_abs .+ m_abs .* xx)
f

In [ ]:
function gaussian_log_likelihood(x, mean, sigma)
    return sum(-log.(sigma * sqrt(2*π)) - 0.5 * (x - mean).^2 ./ sigma.^2)
end

In [ ]:
gaussian_log_likelihood(data.y, m .* data.x .+ b, data.sigma_y)

In [ ]:
function objective(params)
    b = params[1]
    m = params[2]
    y_pred = b .+ m .* data.x
    return -gaussian_log_likelihood(data.y .+ 0., y_pred, data.sigma_y .+ 0.)
end

In [ ]:
[b,m].+0.

In [ ]:
result = optimize(objective, [b,m] .+ 0.)

In [ ]:
b_opt,m_opt = Optim.minimizer(result)

In [ ]:
function objective_2(x, b, m, y, sigma_y)
    y_pred = b .+ m .* x
    return -gaussian_log_likelihood(y, y_pred, sigma_y)
end

In [ ]:
result = optimize(p -> objective_2(data.x, p[1], p[2], data.y, data.sigma_y),
                  [b,m] .+ 0.);
Optim.converged(result)

In [ ]:
b_opt,m_opt = Optim.minimizer(result)

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :black)
lines!(xx, b_opt .+ m_opt .* xx, )
lines!(xx, b_abs .+ m_abs .* xx, color=:red)
f

In [ ]:
append!([1,2],[3,4])

In [ ]:
data.x[1:0]

In [ ]:
n = length(data.x)

B_jack = zeros(n)
M_jack = zeros(n)

for i in 1:n
    x_sub = append!(data.x[1:(i-1)], data.x[(i+1):n])
    y_sub = append!(data.y[1:(i-1)], data.y[(i+1):n])
    s_sub = append!(data.sigma_y[1:(i-1)], data.sigma_y[(i+1):n])
    result = optimize(p -> objective_2(x_sub, p[1], p[2], y_sub, s_sub),
                      [b,m] .+ 0.);
    @assert(Optim.converged(result))
    b,m = Optim.minimizer(result)
    B_jack[i] = b
    M_jack[i] = m
end

In [ ]:
M_jack

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :black)
for i in 1:n
    lines!(xx, B_jack[i] .+ M_jack[i] .* xx, color=:grey)
end
lines!(xx, b_opt .+ m_opt .* xx, color=:red, linewidth=3)
f

In [ ]:
f = Figure()
Axis(f[1, 1])
scatter!(B_jack, M_jack)
f

In [ ]:
using Statistics

In [ ]:
m_jack = mean(M_jack)
b_jack = mean(B_jack)
b_jack, m_jack

In [ ]:
sigma_m_jack = (n-1)/n * std(M_jack .- m_jack)

In [ ]:
sigma_b_jack = (n-1)/n * std(B_jack .- b_jack)

In [ ]:
f = Figure(resolution=(800,500))
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :black)

lines!(xx, (b_jack .+ sigma_b_jack) .+ m_jack .* xx, color=:grey)
lines!(xx, (b_jack .- sigma_b_jack) .+ m_jack .* xx, color=:grey)
lines!(xx, b_jack .+ (m_jack .+ sigma_m_jack) .* xx, color=:grey)
lines!(xx, b_jack .+ (m_jack .- sigma_m_jack) .* xx, color=:grey)

lines!(xx, b_opt .+ m_opt .* xx, color=:red)#, linewidth=3)
f